In [60]:
from pulp import *

In [61]:
prob = LpProblem("TSP", LpMinimize)

In [62]:
departures = ['C', 'N', 'W', 'A']
arrivals = ['C', 'N', 'W', 'A']

travel_costs = {
    'C': {'N':10,'W':25,'A':8},
    'N': {'C':7, 'W': 20},
    'W': {'N': 19, 'C':26, 'A':8},
    'A': {'W':5, 'C':6}
}



In [63]:
# departures = ['C', 'A']
# arrivals = ['C', 'A']

# travel_costs = {
#     'C': {'A':8},
#     'A': {'C':6}
# }



In [64]:
# vists = LpVariable.dicts(name='trip_starts', indices=travel_costs, lowBound=0, upBound=1, cat=LpInteger)

# x = pulp.LpVariable.dicts(
#     "table", possible_tables, lowBound=0, upBound=1, cat=pulp.LpInteger
# )

# classmethoddicts(name, indices=None, lowBound=None, upBound=None, cat='Continuous', indexStart=[])

In [65]:
routes = [ (d, a) for d in departures for a in arrivals if a in travel_costs[d]]

In [66]:
routes

[('C', 'N'),
 ('C', 'W'),
 ('C', 'A'),
 ('N', 'C'),
 ('N', 'W'),
 ('W', 'C'),
 ('W', 'N'),
 ('W', 'A'),
 ('A', 'C'),
 ('A', 'W')]

In [67]:
variables = LpVariable.dicts("best route", routes, 0, 1, LpInteger)

In [68]:
variables

{('C', 'N'): best_route_('C',_'N'),
 ('C', 'W'): best_route_('C',_'W'),
 ('C', 'A'): best_route_('C',_'A'),
 ('N', 'C'): best_route_('N',_'C'),
 ('N', 'W'): best_route_('N',_'W'),
 ('W', 'C'): best_route_('W',_'C'),
 ('W', 'N'): best_route_('W',_'N'),
 ('W', 'A'): best_route_('W',_'A'),
 ('A', 'C'): best_route_('A',_'C'),
 ('A', 'W'): best_route_('A',_'W')}

In [69]:
prob += (
    lpSum([variables[(o,d)] * travel_costs[o][d] for (o, d) in routes]),
    "sum_cost", )

In [70]:
prob

TSP:
MINIMIZE
6*best_route_('A',_'C') + 5*best_route_('A',_'W') + 8*best_route_('C',_'A') + 10*best_route_('C',_'N') + 25*best_route_('C',_'W') + 7*best_route_('N',_'C') + 20*best_route_('N',_'W') + 8*best_route_('W',_'A') + 26*best_route_('W',_'C') + 19*best_route_('W',_'N') + 0
VARIABLES
0 <= best_route_('A',_'C') <= 1 Integer
0 <= best_route_('A',_'W') <= 1 Integer
0 <= best_route_('C',_'A') <= 1 Integer
0 <= best_route_('C',_'N') <= 1 Integer
0 <= best_route_('C',_'W') <= 1 Integer
0 <= best_route_('N',_'C') <= 1 Integer
0 <= best_route_('N',_'W') <= 1 Integer
0 <= best_route_('W',_'A') <= 1 Integer
0 <= best_route_('W',_'C') <= 1 Integer
0 <= best_route_('W',_'N') <= 1 Integer

In [71]:
# Constraints of the problem
for a in arrivals:
    prob += (
            lpSum([variables[(d,a)] for d in departures if  (d,a) in variables]) == 1,
            f"Sum_Arrivel_{a}",
            )

for d in departures:
    prob += (
            lpSum([variables[(d,a)]  for a in arrivals if (d,a) in variables]) == 1,
            f"Sum_departure_{d}",
            )

In [72]:
# Subtour elimination: Adding helper variables to prevent subtours
u = LpVariable.dicts('u', departures, lowBound=0, upBound=len(departures)-1, cat=LpInteger)

In [73]:
for i in departures:
    for j in arrivals:
        if i != j and (i, j) in routes:
            prob += u[i] - u[j] + len(departures) * variables[(i, j)] <= len(departures) - 1, f"SubtourElimination_{i}_{j}"

In [74]:
prob

TSP:
MINIMIZE
6*best_route_('A',_'C') + 5*best_route_('A',_'W') + 8*best_route_('C',_'A') + 10*best_route_('C',_'N') + 25*best_route_('C',_'W') + 7*best_route_('N',_'C') + 20*best_route_('N',_'W') + 8*best_route_('W',_'A') + 26*best_route_('W',_'C') + 19*best_route_('W',_'N') + 0
SUBJECT TO
Sum_Arrivel_C: best_route_('A',_'C') + best_route_('N',_'C')
 + best_route_('W',_'C') = 1

Sum_Arrivel_N: best_route_('C',_'N') + best_route_('W',_'N') = 1

Sum_Arrivel_W: best_route_('A',_'W') + best_route_('C',_'W')
 + best_route_('N',_'W') = 1

Sum_Arrivel_A: best_route_('C',_'A') + best_route_('W',_'A') = 1

Sum_departure_C: best_route_('C',_'A') + best_route_('C',_'N')
 + best_route_('C',_'W') = 1

Sum_departure_N: best_route_('N',_'C') + best_route_('N',_'W') = 1

Sum_departure_W: best_route_('W',_'A') + best_route_('W',_'C')
 + best_route_('W',_'N') = 1

Sum_departure_A: best_route_('A',_'C') + best_route_('A',_'W') = 1

SubtourElimination_C_N: 4 best_route_('C',_'N') + u_C - u_N <= 3

Subtou

In [75]:
prob.writeLP("TSP.lp")

[best_route_('A',_'C'),
 best_route_('A',_'W'),
 best_route_('C',_'A'),
 best_route_('C',_'N'),
 best_route_('C',_'W'),
 best_route_('N',_'C'),
 best_route_('N',_'W'),
 best_route_('W',_'A'),
 best_route_('W',_'C'),
 best_route_('W',_'N'),
 u_A,
 u_C,
 u_N,
 u_W]

In [76]:
prob.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/opc/.pyenv/versions/3.8.0/envs/label/lib/python3.8/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/f04f4521ba1c4500b0fdfbec6337e094-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /tmp/f04f4521ba1c4500b0fdfbec6337e094-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 23 COLUMNS
At line 112 RHS
At line 131 BOUNDS
At line 146 ENDATA
Problem MODEL has 18 rows, 14 columns and 50 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 34.5 - 0.00 seconds
Cgl0003I 2 fixed, 4 tightened bounds, 5 strengthened rows, 2 substitutions
Cgl0003I 2 fixed, 2 tightened bounds, 0 strengthened rows, 0 substitutions
Cgl0000I Cut generators found to be infeasible! (or unbounded)
Pre-processing says infeasible or unbounded
Option for printingOptions changed from normal to all
Total time (CPU sec

-1

In [77]:
print("Status:", LpStatus[prob.status])

for v in prob.variables():
    if v.varValue > 0:
        print(v.name, "=", v.varValue)

print("Total distance = ", value(prob.objective))

Status: Infeasible
best_route_('A',_'W') = 1.0
best_route_('C',_'A') = 0.5
best_route_('C',_'N') = 0.5
best_route_('N',_'C') = 1.0
best_route_('W',_'A') = 0.5
best_route_('W',_'N') = 0.5
u_C = 1.0
u_W = 1.0
Total distance =  34.5


In [78]:
from pulp import *

# Problem definition
prob = LpProblem("TSP", LpMinimize)
departures = ['C', 'N', 'W', 'A']
arrivals = ['C', 'N', 'W', 'A']

# Travel costs between cities
travel_costs = {
    'C': {'N':10,'W':25,'A':8},
    'N': {'C':7, 'W': 20},
    'W': {'N': 19, 'C':26, 'A':8},
    'A': {'W':5, 'C':6}
}

# Create routes that exist in the travel_costs dictionary
routes = [(d, a) for d in departures for a in arrivals if a in travel_costs[d]]

# Define decision variables for each route (binary decision variables)
variables = LpVariable.dicts("best_route", routes, 0, 1, LpBinary)

# Objective function: minimize the total travel cost
prob += lpSum([variables[(o, d)] * travel_costs[o][d] for (o, d) in routes]), "sum_cost"

# Constraints: Each city must be visited once (both for arrivals and departures)
# For arrivals
for a in arrivals:
    prob += lpSum([variables[(d, a)] for d in departures if (d, a) in variables]) == 1, f"Sum_Arrival_{a}"

# For departures
for d in departures:
    prob += lpSum([variables[(d, a)] for a in arrivals if (d, a) in variables]) == 1, f"Sum_Departure_{d}"

# Subtour elimination: Adding helper variables to prevent subtours
# Set lower bound to 1, since u[i] represents the order in the tour
u = LpVariable.dicts('u', departures, lowBound=0, upBound=len(departures)-1, cat=LpInteger)

# Subtour elimination constraints using MTZ formulation
for i in departures:
    for j in arrivals:
        if i != j and (i, j) in routes and j!=0:
            prob += u[i] - u[j] + len(departures) * variables[(i, j)] <= len(departures) - 1, f"SubtourElimination_{i}_{j}"

# Solve the problem
prob.solve()

# Output results
print(f"Status: {LpStatus[prob.status]}")
for v in prob.variables():
    print(f"{v.name} = {v.varValue}")


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/opc/.pyenv/versions/3.8.0/envs/label/lib/python3.8/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/c4aed2489a764811bce04dd2478b68b8-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /tmp/c4aed2489a764811bce04dd2478b68b8-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 23 COLUMNS
At line 112 RHS
At line 131 BOUNDS
At line 146 ENDATA
Problem MODEL has 18 rows, 14 columns and 50 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 34.5 - 0.00 seconds
Cgl0003I 2 fixed, 4 tightened bounds, 5 strengthened rows, 2 substitutions
Cgl0003I 2 fixed, 2 tightened bounds, 0 strengthened rows, 0 substitutions
Cgl0000I Cut generators found to be infeasible! (or unbounded)
Pre-processing says infeasible or unbounded
Option for printingOptions changed from normal to all
Total time (CPU sec

In [79]:
prob

TSP:
MINIMIZE
6*best_route_('A',_'C') + 5*best_route_('A',_'W') + 8*best_route_('C',_'A') + 10*best_route_('C',_'N') + 25*best_route_('C',_'W') + 7*best_route_('N',_'C') + 20*best_route_('N',_'W') + 8*best_route_('W',_'A') + 26*best_route_('W',_'C') + 19*best_route_('W',_'N') + 0
SUBJECT TO
Sum_Arrival_C: best_route_('A',_'C') + best_route_('N',_'C')
 + best_route_('W',_'C') = 1

Sum_Arrival_N: best_route_('C',_'N') + best_route_('W',_'N') = 1

Sum_Arrival_W: best_route_('A',_'W') + best_route_('C',_'W')
 + best_route_('N',_'W') = 1

Sum_Arrival_A: best_route_('C',_'A') + best_route_('W',_'A') = 1

Sum_Departure_C: best_route_('C',_'A') + best_route_('C',_'N')
 + best_route_('C',_'W') = 1

Sum_Departure_N: best_route_('N',_'C') + best_route_('N',_'W') = 1

Sum_Departure_W: best_route_('W',_'A') + best_route_('W',_'C')
 + best_route_('W',_'N') = 1

Sum_Departure_A: best_route_('A',_'C') + best_route_('A',_'W') = 1

SubtourElimination_C_N: 4 best_route_('C',_'N') + u_C - u_N <= 3

Subtou